In [18]:
import numpy as np
import os
import tensorflow as tf

print("Loading Data")
with open("data/small_vocab_en.txt", "r") as f:
    en = f.read().split("\n")
with open("data/small_vocab_fr.txt", "r", encoding="UTF-8") as f:
    fr = f.read().split("\n")

print("Creating Tokenizers")
en_tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=False)
fr_tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=False)

Loading Data
Creating Tokenizers


In [19]:
    print("Tokenizing English")
    en_tokenizer.fit_on_texts(en)
    en_tokenized = en_tokenizer.texts_to_sequences(en)

Tokenizing English


In [20]:
print(en[0])
print(en_tokenized[0])

new jersey is sometimes quiet during autumn , and it is snowy in april .
[17, 23, 1, 8, 67, 4, 39, 7, 3, 1, 55, 2, 44]


In [21]:
    print("Tokenizing French")
    fr_tokenizer.fit_on_texts(fr)
    fr_tokenized = fr_tokenizer.texts_to_sequences(fr)

Tokenizing French


In [22]:
print(fr[0])
print(fr_tokenized[0])

new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
[35, 34, 1, 8, 67, 37, 11, 24, 6, 3, 1, 112, 2, 50]


In [23]:
print("Padding Tokens")
en_max_length = max([len(x) for x in en_tokenized])
en_padded = tf.keras.preprocessing.sequence.pad_sequences(en_tokenized, maxlen = en_max_length, padding = 'post')
fr_max_length = max([len(x) for x in fr_tokenized])
fr_padded = tf.keras.preprocessing.sequence.pad_sequences(fr_tokenized, maxlen = fr_max_length, padding = 'post')

Padding Tokens


In [24]:
en_padded = en_padded.reshape(*en_padded.shape, 1)
en_padded.shape
fr_padded = fr_padded.reshape(*fr_padded.shape, 1)
fr_padded.shape

(137861, 21, 1)

In [25]:
len(fr_tokenizer.word_index)+1

345

In [26]:
print(en_padded.shape)
print(fr_padded.shape)
en_padded = tf.keras.preprocessing.sequence.pad_sequences(en_padded, fr_max_length)
en_padded = en_padded.reshape((-1, fr_padded.shape[-2], 1))
print(en_padded.shape)

(137861, 15, 1)
(137861, 21, 1)
(137861, 21, 1)


In [27]:
input_shape = shape=en_padded.shape[1:]
input_layer = tf.keras.Input(shape=input_shape)
rnn1 = tf.keras.layers.SimpleRNN(units=64, return_sequences=True)(input_layer)
rnn2 = tf.keras.layers.SimpleRNN(units=64, return_sequences=True)(rnn1)
output = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(len(fr_tokenizer.word_index)+2))(rnn2)
en_padded = en_padded.astype("float64")
model = tf.keras.Model(input_layer, tf.keras.layers.Activation('softmax')(output))
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(1e-3), metrics=['accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 21, 1)]           0         
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 21, 64)            4224      
                                                                 
 simple_rnn_6 (SimpleRNN)    (None, 21, 64)            8256      
                                                                 
 time_distributed_1 (TimeDis  (None, 21, 346)          22490     
 tributed)                                                       
                                                                 
 activation_1 (Activation)   (None, 21, 346)           0         
                                                                 
Total params: 34,970
Trainable params: 34,970
Non-trainable params: 0
_______________________________________________________

In [17]:
inputs = np.random.random((32, 10, 8))
simple_rnn = tf.keras.layers.SimpleRNN(1)
simple_rnn(inputs)

<tf.Tensor: shape=(32, 1), dtype=float32, numpy=
array([[-0.7924938 ],
       [-0.67908573],
       [-0.92488134],
       [-0.61848885],
       [-0.2698248 ],
       [-0.66829515],
       [-0.6452007 ],
       [-0.60497856],
       [-0.6633147 ],
       [-0.8222345 ],
       [-0.8668877 ],
       [-0.74706286],
       [-0.92752016],
       [-0.8825594 ],
       [-0.9231137 ],
       [-0.5727092 ],
       [-0.63355345],
       [-0.7772994 ],
       [-0.8308711 ],
       [-0.7867554 ],
       [-0.9032331 ],
       [-0.89143157],
       [-0.82360923],
       [-0.57804286],
       [-0.69223523],
       [-0.90722555],
       [-0.47508335],
       [-0.79781204],
       [-0.8052938 ],
       [-0.76453435],
       [-0.8579754 ],
       [-0.6569055 ]], dtype=float32)>

In [207]:
print(len(en_tokenizer.word_index.keys()))

199


In [28]:

model.fit(en_padded, fr_padded, batch_size=16, epochs=10, validation_split=.2)

Epoch 1/10
6893/6893 [==============================] - 33s 5ms/step - loss: 1.9672 - accuracy: 0.5268 - val_loss: 1.8541 - val_accuracy: 0.5399
Epoch 2/10
6893/6893 [==============================] - 33s 5ms/step - loss: 1.8451 - accuracy: 0.5385 - val_loss: 1.8321 - val_accuracy: 0.5417
Epoch 3/10
6893/6893 [==============================] - 34s 5ms/step - loss: 1.8286 - accuracy: 0.5411 - val_loss: 1.8150 - val_accuracy: 0.5439
Epoch 4/10
6893/6893 [==============================] - 33s 5ms/step - loss: 1.8210 - accuracy: 0.5418 - val_loss: 1.8201 - val_accuracy: 0.5417
Epoch 5/10
6893/6893 [==============================] - 33s 5ms/step - loss: 1.8177 - accuracy: 0.5421 - val_loss: 1.8076 - val_accuracy: 0.5444
Epoch 6/10
6893/6893 [==============================] - 37s 5ms/step - loss: 1.8150 - accuracy: 0.5424 - val_loss: 1.8061 - val_accuracy: 0.5450
Epoch 7/10
6893/6893 [==============================] - 34s 5ms/step - loss: 1.8122 - accuracy: 0.5428 - val_loss: 1.8051 - val_ac

In [29]:
model.evaluate(en_padded, fr_padded)

4309/4309 [==============================] - 10s 2ms/step - loss: 1.7981 - accuracy: 0.5444


[1.7981255054473877, 0.5443678498268127]

In [31]:
t = model(en_padded[0])

In [210]:
input_shape = shape=en_padded.shape[1:]
input_layer = tf.keras.Input(shape=input_shape)
rnn1 = tf.keras.layers.LSTM(units=64, return_sequences=True)(input_layer)
rnn2 = tf.keras.layers.LSTM(units=64, return_sequences=True)(rnn1)
output = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(len(fr_tokenizer.word_index)+2))(rnn2)
en_padded = en_padded.astype("float64")
model = tf.keras.Model(input_layer, tf.keras.layers.Activation('softmax')(output))
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(1e-3), metrics=['accuracy'])
model.summary()

Model: "model_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_28 (InputLayer)       [(None, 21, 1)]           0         
                                                                 
 lstm (LSTM)                 (None, 21, 64)            16896     
                                                                 
 lstm_1 (LSTM)               (None, 21, 64)            33024     
                                                                 
 time_distributed_24 (TimeDi  (None, 21, 346)          22490     
 stributed)                                                      
                                                                 
 activation_8 (Activation)   (None, 21, 346)           0         
                                                                 
Total params: 72,410
Trainable params: 72,410
Non-trainable params: 0
______________________________________________________

In [211]:
model.fit(en_padded, fr_padded, batch_size=16, epochs=10, validation_split=.2)

Epoch 1/10
6893/6893 [==============================] - 56s 8ms/step - loss: 2.0250 - accuracy: 0.5207 - val_loss: 1.8652 - val_accuracy: 0.5390
Epoch 2/10
6893/6893 [==============================] - 57s 8ms/step - loss: 1.8514 - accuracy: 0.5383 - val_loss: 1.8326 - val_accuracy: 0.5430
Epoch 3/10
6893/6893 [==============================] - 60s 9ms/step - loss: 1.8253 - accuracy: 0.5416 - val_loss: 1.8103 - val_accuracy: 0.5452
Epoch 4/10
6893/6893 [==============================] - 64s 9ms/step - loss: 1.8121 - accuracy: 0.5433 - val_loss: 1.8160 - val_accuracy: 0.5429
Epoch 5/10
6893/6893 [==============================] - 74s 11ms/step - loss: 1.8076 - accuracy: 0.5437 - val_loss: 1.8118 - val_accuracy: 0.5442
Epoch 6/10
6893/6893 [==============================] - 75s 11ms/step - loss: 1.8044 - accuracy: 0.5439 - val_loss: 1.7999 - val_accuracy: 0.5452
Epoch 7/10
6893/6893 [==============================] - 76s 11ms/step - loss: 1.8020 - accuracy: 0.5440 - val_loss: 1.7977 - val

In [212]:
model.evaluate(en_padded, fr_padded)

4309/4309 [==============================] - 22s 5ms/step - loss: 1.7895 - accuracy: 0.5455


[1.7894706726074219, 0.5455111861228943]

In [214]:
input_shape = shape=en_padded.shape[1:]
input_layer = tf.keras.Input(shape=input_shape)
rnn1 = tf.keras.layers.GRU(units=64, return_sequences=True)(input_layer)
rnn2 = tf.keras.layers.GRU(units=64, return_sequences=True)(rnn1)
output = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(len(fr_tokenizer.word_index)+2))(rnn2)
en_padded = en_padded.astype("float64")
model = tf.keras.Model(input_layer, tf.keras.layers.Activation('softmax')(output))
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(1e-3), metrics=['accuracy'])
model.summary()

Model: "model_38"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_30 (InputLayer)       [(None, 21, 1)]           0         
                                                                 
 gru_2 (GRU)                 (None, 21, 64)            12864     
                                                                 
 gru_3 (GRU)                 (None, 21, 64)            24960     
                                                                 
 time_distributed_26 (TimeDi  (None, 21, 346)          22490     
 stributed)                                                      
                                                                 
 activation_10 (Activation)  (None, 21, 346)           0         
                                                                 
Total params: 60,314
Trainable params: 60,314
Non-trainable params: 0
______________________________________________________

In [215]:
model.fit(en_padded, fr_padded, batch_size=16, epochs=10, validation_split=.2)

Epoch 1/10
6893/6893 [==============================] - 66s 9ms/step - loss: 1.9876 - accuracy: 0.5258 - val_loss: 1.8442 - val_accuracy: 0.5404
Epoch 2/10
6893/6893 [==============================] - 71s 10ms/step - loss: 1.8372 - accuracy: 0.5394 - val_loss: 1.8188 - val_accuracy: 0.5414
Epoch 3/10
6893/6893 [==============================] - 69s 10ms/step - loss: 1.8166 - accuracy: 0.5426 - val_loss: 1.8048 - val_accuracy: 0.5456
Epoch 4/10
6893/6893 [==============================] - 67s 10ms/step - loss: 1.8110 - accuracy: 0.5428 - val_loss: 1.8204 - val_accuracy: 0.5432
Epoch 5/10
6893/6893 [==============================] - 70s 10ms/step - loss: 1.8117 - accuracy: 0.5421 - val_loss: 1.8040 - val_accuracy: 0.5456
Epoch 6/10
6893/6893 [==============================] - 70s 10ms/step - loss: 1.8039 - accuracy: 0.5433 - val_loss: 1.8081 - val_accuracy: 0.5449
Epoch 7/10
6893/6893 [==============================] - 70s 10ms/step - loss: 1.8027 - accuracy: 0.5435 - val_loss: 1.8105 - 

In [216]:
model.evaluate(en_padded, fr_padded)

4309/4309 [==============================] - 25s 6ms/step - loss: 1.7941 - accuracy: 0.5446


[1.7941335439682007, 0.5445868372917175]